<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-10-22 15:57:43


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['Criteria'].isin(['V40','V40N'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] > 70) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [7]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2022-11-15 00:00:00+05:30,2240.0,1943.0,15.30,AKZOINDIA.NS,2899.85,2050.65,9.24,73.0,-1.0,Sell,V40
2022-11-15 00:00:00+05:30,1056.0,956.0,10.49,HCLTECH.NS,1292.64,982.26,7.54,85.0,-1.0,Sell,V40
2022-11-18 00:00:00+05:30,394.0,447.0,-11.88,SIS.NS,605.18,320.65,22.84,28.0,1.0,Buy,V40N
2022-11-21 00:00:00+05:30,564.0,447.0,26.15,FINCABLES.NS,1195.55,506.19,11.39,76.0,-1.0,Sell,V40N
2022-11-25 00:00:00+05:30,20019.0,18208.0,9.95,ABBOTINDIA.NS,24555.80,18130.57,10.41,71.0,-1.0,Sell,V40
...,...,...,...,...,...,...,...,...,...,...,...
2023-09-08 00:00:00+05:30,39025.0,39790.0,-1.92,HONAUT.NS,48731.02,34410.93,13.41,29.0,1.0,Buy,V40N
2023-09-25 00:00:00+05:30,2340.0,2474.0,-5.40,RELIANCE.NS,2831.85,2193.50,6.70,29.0,1.0,Buy,V40
2023-10-12 00:00:00+05:30,1359.0,1539.0,-11.72,LUXIND.NS,4531.14,1125.23,20.75,28.0,1.0,Buy,V40N


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-06-13 00:00:00+05:30,1210.0,1201.0,0.79,SFL.NS,1972.20,951.40,27.23,81.0,-1.0,Sell,V40N
2023-06-16 00:00:00+05:30,1604.0,1577.0,1.69,BATAINDIA.NS,2155.97,1376.34,16.52,71.0,-1.0,Sell,V40
2023-06-19 00:00:00+05:30,716.0,670.0,6.82,MOTILALOFS.NS,1090.66,551.30,29.82,72.0,-1.0,Sell,V40N
2023-06-20 00:00:00+05:30,2212.0,2128.0,3.94,LALPATHLAB.NS,4164.29,1770.80,24.91,76.0,-1.0,Sell,V40N
2023-06-23 00:00:00+05:30,4834.0,4723.0,2.36,GILLETTE.NS,7529.30,4186.20,15.48,74.0,-1.0,Sell,V40
2023-06-26 00:00:00+05:30,14479.0,13857.0,4.49,PGHH.NS,18367.60,13311.20,8.77,73.0,-1.0,Sell,V40
2023-06-30 00:00:00+05:30,699.0,647.0,8.02,ERIS.NS,908.25,567.95,23.06,73.0,-1.0,Sell,V40N
2023-07-04 00:00:00+05:30,4510.0,4357.0,3.52,BAYERCROP.NS,5923.63,3918.08,15.12,77.0,-1.0,Sell,V40N
2023-07-11 00:00:00+05:30,471.0,447.0,5.34,SUNTV.NS,661.80,393.46,19.67,71.0,-1.0,Sell,V40N


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2022-11-18 00:00:00+05:30,394.0,447.0,-11.88,SIS.NS,605.18,320.65,22.84,28.0,1.0,Buy,V40N
2023-02-27 00:00:00+05:30,660.0,708.0,-6.80,RAJESHEXPO.NS,949.35,436.95,51.01,28.0,1.0,Buy,V40N
2023-04-10 00:00:00+05:30,960.0,980.0,-2.02,SYMPHONY.NS,1466.54,821.23,16.92,25.0,1.0,Buy,V40N
2023-07-06 00:00:00+05:30,3187.0,3266.0,-2.42,EICHERMOT.NS,3808.41,2821.26,12.96,27.0,1.0,Buy,V40N
2023-08-18 00:00:00+05:30,1751.0,1843.0,-5.00,KOTAKBANK.NS,2207.80,1646.89,6.31,28.0,1.0,Buy,V40
2023-08-25 00:00:00+05:30,1562.0,1598.0,-2.28,HDFCBANK.NS,1728.20,1434.45,8.86,27.0,1.0,Buy,V40
2023-09-08 00:00:00+05:30,39025.0,39790.0,-1.92,HONAUT.NS,48731.02,34410.93,13.41,29.0,1.0,Buy,V40N
2023-09-25 00:00:00+05:30,2340.0,2474.0,-5.40,RELIANCE.NS,2831.85,2193.50,6.70,29.0,1.0,Buy,V40
2023-10-12 00:00:00+05:30,1359.0,1539.0,-11.72,LUXIND.NS,4531.14,1125.23,20.75,28.0,1.0,Buy,V40N


In [10]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [11]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)